In [ ]:
# Install packages
!pip install opencv-python
!pip install shapely
!pip install geopandas
!pip install rasterio
!pip install pandas
!pip install matplotlib
!pip install scipy

# Import modules
import os
import shutil
from glob import glob
import numpy as np
import pandas as pd
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.warp import transform, transform_bounds
from shapely.geometry import box
import geopandas as gpd
import cv2
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt

In [ ]:
# Define base directory and years
base_dir = "D:/Sushrut"
years = ["2020", "2021", "2022", "2023", "2024"]
scale_factor = 0.001  # MODIS scale factor
fill_value = -28672  # MODIS fill value
valid_range = (-100, 5000)  # Valid AOD range

# Process each year
for year in years:
    print(f"\n Processing Year: {year}...")

    aod_folder = os.path.join(base_dir, year, "AOD_047")
    qa_folder = os.path.join(base_dir, year, "QA")
    output_folder = os.path.join(base_dir, year, "Filtered_AOD_047")
    os.makedirs(output_folder, exist_ok=True)

    # Find all AOD files
    aod_files = sorted(glob(os.path.join(aod_folder, "*.tif")))

    if not aod_files:
        print(f"️ No AOD files found in {aod_folder}. Skipping...")
        continue

    for aod_file in aod_files:
        # Extract date from filename (e.g., A2024001)
        date_str = os.path.basename(aod_file).split(".")[1]

        # Find corresponding QA file
        qa_files = glob(os.path.join(qa_folder, f"*{date_str}*AOD_QA*.tif"))
        if not qa_files:
            print(f"️ No QA file found for {date_str} in {year}. Skipping...")
            continue
        qa_file = qa_files[0]  # Select the first match

        # Read AOD file
        with rasterio.open(aod_file) as src:
            aod_data = src.read(1).astype(np.float32)
            meta = src.meta

        # Handle fill values (-28672 → NaN)
        aod_data[aod_data == fill_value] = np.nan

        # Apply scale factor only to valid AOD values (-100 to 5000)
        valid_mask = (aod_data >= valid_range[0]) & (aod_data <= valid_range[1])
        aod_data = np.where(valid_mask, aod_data * scale_factor, np.nan)

        # Read QA file
        with rasterio.open(qa_file) as src:
            qa_data = src.read(1).astype(np.uint16)

        # Apply QA Masking (Keep Best, Moderate & Lower Confidence Pixels)
        qa_bits = np.bitwise_and(qa_data, 3)  # Extract bits 0-1 (AOD Quality)
        qa_valid_mask = (qa_bits == 3) | (qa_bits == 2) | (qa_bits == 1)

        # Apply QA mask
        filtered_data = np.where(qa_valid_mask, aod_data, np.nan)

        # Save filtered AOD file
        output_file = os.path.join(output_folder, f"Filtered_AOD_{date_str}.tif")
        meta.update(dtype=rasterio.float32, nodata=np.nan)

        with rasterio.open(output_file, "w", **meta) as dst:
            dst.write(filtered_data, 1)

        print(f" Filtered AOD file saved for {date_str} in {year}: {output_file}")

print("\n QA Masking Complete for All Years!")



📌 Processing Year: 2020...
✅ Filtered AOD file saved for A2020001 in 2020: D:/Sushrut\2020\Filtered_AOD_047\Filtered_AOD_A2020001.tif
✅ Filtered AOD file saved for A2020001 in 2020: D:/Sushrut\2020\Filtered_AOD_047\Filtered_AOD_A2020001.tif
✅ Filtered AOD file saved for A2020001 in 2020: D:/Sushrut\2020\Filtered_AOD_047\Filtered_AOD_A2020001.tif
✅ Filtered AOD file saved for A2020002 in 2020: D:/Sushrut\2020\Filtered_AOD_047\Filtered_AOD_A2020002.tif
✅ Filtered AOD file saved for A2020002 in 2020: D:/Sushrut\2020\Filtered_AOD_047\Filtered_AOD_A2020002.tif
✅ Filtered AOD file saved for A2020002 in 2020: D:/Sushrut\2020\Filtered_AOD_047\Filtered_AOD_A2020002.tif
✅ Filtered AOD file saved for A2020002 in 2020: D:/Sushrut\2020\Filtered_AOD_047\Filtered_AOD_A2020002.tif
✅ Filtered AOD file saved for A2020003 in 2020: D:/Sushrut\2020\Filtered_AOD_047\Filtered_AOD_A2020003.tif
✅ Filtered AOD file saved for A2020003 in 2020: D:/Sushrut\2020\Filtered_AOD_047\Filtered_AOD_A2020003.tif
✅ Filtere

In [ ]:

# Define base directory and years
base_dir = "D:/Sushrut"
years = ["2020", "2021", "2022", "2023", "2024"]

# Process each year
for year in years:
    print(f"\n Processing Year: {year}...")

    filtered_folder = os.path.join(base_dir, year, "Filtered_AOD_047")
    output_folder = os.path.join(base_dir, year, "Merged_AOD_047")
    os.makedirs(output_folder, exist_ok=True)

    # Find all filtered AOD files
    filtered_files = sorted(glob(os.path.join(filtered_folder, "*.tif")))

    if not filtered_files:
        print(f"️ No filtered AOD files found in {filtered_folder}. Skipping...")
        continue

    # Group files by date
    file_dict = {}
    for file in filtered_files:
        date_str = os.path.basename(file).split("_")[2].split(".")[0]  # FIX: Remove extra ".tif"
        if date_str not in file_dict:
            file_dict[date_str] = []
        file_dict[date_str].append(file)

    # Process each date
    for date, file_list in file_dict.items():
        print(f" Merging {len(file_list)} AOD files for {date}...")

        # Open first file to get metadata
        with rasterio.open(file_list[0]) as src:
            meta = src.meta
            meta.update(dtype=rasterio.float32, nodata=np.nan)
            height, width = src.shape

        # Create an empty stack to hold all AOD files for this date
        data_stack = np.full((len(file_list), height, width), np.nan, dtype=np.float32)

        # Load all AOD files for this date
        for i, file in enumerate(file_list):
            with rasterio.open(file) as src:
                data_stack[i] = src.read(1)  # Read band 1

        # Merge by taking the first valid value (or mean if needed)
        merged_data = np.nanmean(data_stack, axis=0)  # Mean of available data

        # Save merged file
        output_file = os.path.join(output_folder, f"Merged_AOD_{date}.tif")
        with rasterio.open(output_file, "w", **meta) as dst:
            dst.write(merged_data.astype(rasterio.float32), 1)

        print(f" Merged AOD file saved for {date}: {output_file}")

print("\n Merging Complete for All Years!")


📌 Processing Year: 2020...
🔄 Merging 1 AOD files for A2020001...
✅ Merged AOD file saved for A2020001: D:/Sushrut\2020\Merged_AOD_047\Merged_AOD_A2020001.tif
🔄 Merging 1 AOD files for A2020002...
✅ Merged AOD file saved for A2020002: D:/Sushrut\2020\Merged_AOD_047\Merged_AOD_A2020002.tif
🔄 Merging 1 AOD files for A2020003...
✅ Merged AOD file saved for A2020003: D:/Sushrut\2020\Merged_AOD_047\Merged_AOD_A2020003.tif
🔄 Merging 1 AOD files for A2020004...
✅ Merged AOD file saved for A2020004: D:/Sushrut\2020\Merged_AOD_047\Merged_AOD_A2020004.tif
🔄 Merging 1 AOD files for A2020005...
✅ Merged AOD file saved for A2020005: D:/Sushrut\2020\Merged_AOD_047\Merged_AOD_A2020005.tif
🔄 Merging 1 AOD files for A2020006...
✅ Merged AOD file saved for A2020006: D:/Sushrut\2020\Merged_AOD_047\Merged_AOD_A2020006.tif
🔄 Merging 1 AOD files for A2020007...
✅ Merged AOD file saved for A2020007: D:/Sushrut\2020\Merged_AOD_047\Merged_AOD_A2020007.tif
🔄 Merging 1 AOD files for A2020008...
✅ Merged AOD file

C:\Users\HP\AppData\Local\Temp\ipykernel_11328\1806635581.py:52: RuntimeWarning: Mean of empty slice
  merged_data = np.nanmean(data_stack, axis=0)  # Mean of available data


✅ Merged AOD file saved for A2020014: D:/Sushrut\2020\Merged_AOD_047\Merged_AOD_A2020014.tif
🔄 Merging 1 AOD files for A2020015...
✅ Merged AOD file saved for A2020015: D:/Sushrut\2020\Merged_AOD_047\Merged_AOD_A2020015.tif
🔄 Merging 1 AOD files for A2020016...
✅ Merged AOD file saved for A2020016: D:/Sushrut\2020\Merged_AOD_047\Merged_AOD_A2020016.tif
🔄 Merging 1 AOD files for A2020017...
✅ Merged AOD file saved for A2020017: D:/Sushrut\2020\Merged_AOD_047\Merged_AOD_A2020017.tif
🔄 Merging 1 AOD files for A2020018...
✅ Merged AOD file saved for A2020018: D:/Sushrut\2020\Merged_AOD_047\Merged_AOD_A2020018.tif
🔄 Merging 1 AOD files for A2020019...
✅ Merged AOD file saved for A2020019: D:/Sushrut\2020\Merged_AOD_047\Merged_AOD_A2020019.tif
🔄 Merging 1 AOD files for A2020020...
✅ Merged AOD file saved for A2020020: D:/Sushrut\2020\Merged_AOD_047\Merged_AOD_A2020020.tif
🔄 Merging 1 AOD files for A2020021...
✅ Merged AOD file saved for A2020021: D:/Sushrut\2020\Merged_AOD_047\Merged_AOD_A20

In [ ]:
from scipy.interpolate import UnivariateSpline
import numpy as np

def apply_spline_smoothing(aod_data, dates, smoothing_factor=1.0):

    smoothed_data = np.full_like(aod_data, np.nan)

    for i in range(aod_data.shape[1]):  # Loop over pixels (columns)
        for j in range(aod_data.shape[2]):  # Loop over pixels (rows)
            pixel_values = aod_data[:, i, j]

            # Find valid (non-NaN) indices
            valid_idx = ~np.isnan(pixel_values)

            if np.sum(valid_idx) > 5:  # At least 5 valid values needed
                x = dates[valid_idx]
                y = pixel_values[valid_idx]

                #  Log-transform to stabilize large variations
                y_log = np.log1p(y)

                #  Apply smoothing spline with adjusted parameters
                spline = UnivariateSpline(x, y_log, s=smoothing_factor)

                #  Compute smoothed values and inverse log-transform
                smoothed_values = np.expm1(spline(dates))

                #  Clip values to realistic range (0 to 5)
                smoothed_data[:, i, j] = np.clip(smoothed_values, 0, 5)

    return smoothed_data


In [ ]:


# Define base directory
base_dir = "D:/Sushrut"
years = ["2020", "2021", "2022", "2023", "2024"]

# Output folders
smoothed_dir = "Smoothed_Trend_AOD"
residual_dir = "Residual_AOD"

# Process each year
for year in years:
    print(f"\n Processing Temporal Gap-Filling for Year: {year}...")

    merged_aod_dir = os.path.join(base_dir, year, "Merged_AOD_047")
    smoothed_aod_dir = os.path.join(base_dir, year, smoothed_dir)
    residual_aod_dir = os.path.join(base_dir, year, residual_dir)

    # Create output folders
    os.makedirs(smoothed_aod_dir, exist_ok=True)
    os.makedirs(residual_aod_dir, exist_ok=True)

    # Get all merged AOD files
    merged_files = sorted([f for f in os.listdir(merged_aod_dir) if f.endswith(".tif")])

    if not merged_files:
        print(f"️ No Merged AOD files found for {year}. Skipping...")
        continue

    # Extract Dates from Filenames
    dates = []
    file_paths = []

    for file in merged_files:
        try:
            date_str = os.path.basename(file).replace("Merged_AOD_A", "").replace(".tif", "")
            dates.append(int(date_str))  # Convert to integer
            file_paths.append(os.path.join(merged_aod_dir, file))
        except Exception as e:
            print(f"️ Skipping file: {file} (Invalid Date Format)")

    # Check if any valid files were found
    if not file_paths:
        print(f" No valid files found for {year}. Skipping...")
        continue

    # Sort by date
    sorted_indices = np.argsort(dates)
    dates = np.array(dates)[sorted_indices]
    file_paths = [file_paths[i] for i in sorted_indices]

    # Read first file to get metadata
    with rasterio.open(file_paths[0]) as src:
        height, width = src.shape
        meta = src.meta
        meta.update(dtype=rasterio.float32, nodata=np.nan)

    # Load all AOD files into memory
    print(" Loading AOD data into memory...")
    aod_data = np.full((len(dates), height, width), np.nan, dtype=np.float32)

    for i, file in enumerate(file_paths):
        with rasterio.open(file) as src:
            aod_data[i] = src.read(1)

    # Process row-wise
    print(" Applying Smoothing Spline...")
    smoothed_data = np.full_like(aod_data, np.nan, dtype=np.float32)
    residuals_data = np.full_like(aod_data, np.nan, dtype=np.float32)

    for i in range(height):
        for j in range(width):
            pixel_values = aod_data[:, i, j]
            valid_idx = ~np.isnan(pixel_values)

            if np.sum(valid_idx) > 10:  # Apply smoothing only if sufficient data exists
                x = dates[valid_idx]
                y = pixel_values[valid_idx]
                spline = UnivariateSpline(x, y, s=0.5)  # Smoothing spline
                smoothed_values = spline(dates)
            else:
                smoothed_values = np.full_like(dates, np.nan, dtype=np.float32)

            # Compute residuals (Original - Smoothed Trend)
            smoothed_data[:, i, j] = smoothed_values
            residuals_data[:, i, j] = pixel_values - smoothed_values

    # Save Smoothed AOD & Residuals as GeoTIFFs
    for k, date in enumerate(dates):
        smoothed_file = os.path.join(smoothed_aod_dir, f"Smoothed_Trend_AOD_A{date}.tif")
        residual_file = os.path.join(residual_aod_dir, f"Residual_AOD_A{date}.tif")

        with rasterio.open(smoothed_file, "w", **meta) as dst:
            dst.write(smoothed_data[k], 1)

        with rasterio.open(residual_file, "w", **meta) as dst:
            dst.write(residuals_data[k], 1)

        print(f" Saved: {smoothed_file}, {residual_file}")

print("\n Temporal Gap-Filling & Smoothing Complete for All Years!")



📌 Processing Temporal Gap-Filling for Year: 2020...
🔄 Loading AOD data into memory...
📈 Applying Smoothing Spline...


C:\Users\HP\AppData\Local\Temp\ipykernel_11328\265298189.py:84: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  spline = UnivariateSpline(x, y, s=0.5)  # Smoothing spline


✅ Saved: D:/Sushrut\2020\Smoothed_Trend_AOD\Smoothed_Trend_AOD_A2020001.tif, D:/Sushrut\2020\Residual_AOD\Residual_AOD_A2020001.tif
✅ Saved: D:/Sushrut\2020\Smoothed_Trend_AOD\Smoothed_Trend_AOD_A2020002.tif, D:/Sushrut\2020\Residual_AOD\Residual_AOD_A2020002.tif
✅ Saved: D:/Sushrut\2020\Smoothed_Trend_AOD\Smoothed_Trend_AOD_A2020003.tif, D:/Sushrut\2020\Residual_AOD\Residual_AOD_A2020003.tif
✅ Saved: D:/Sushrut\2020\Smoothed_Trend_AOD\Smoothed_Trend_AOD_A2020004.tif, D:/Sushrut\2020\Residual_AOD\Residual_AOD_A2020004.tif
✅ Saved: D:/Sushrut\2020\Smoothed_Trend_AOD\Smoothed_Trend_AOD_A2020005.tif, D:/Sushrut\2020\Residual_AOD\Residual_AOD_A2020005.tif
✅ Saved: D:/Sushrut\2020\Smoothed_Trend_AOD\Smoothed_Trend_AOD_A2020006.tif, D:/Sushrut\2020\Residual_AOD\Residual_AOD_A2020006.tif
✅ Saved: D:/Sushrut\2020\Smoothed_Trend_AOD\Smoothed_Trend_AOD_A2020007.tif, D:/Sushrut\2020\Residual_AOD\Residual_AOD_A2020007.tif
✅ Saved: D:/Sushrut\2020\Smoothed_Trend_AOD\Smoothed_Trend_AOD_A2020008.tif,

In [ ]:
#  Define Paths
input_folder = "D:/Sushrut/2024/Residual_AOD"  # Input folder of having Residual_AOD
output_folder = "D:/Sushrut/2024/Cropped_AOD_Files"  # Output folder for cropped files
os.makedirs(output_folder, exist_ok=True)

#  Define Bounding Box
bbox = box(77.8889, 17.0780, 78.7735, 17.8714)
bbox_gdf = gpd.GeoDataFrame({"geometry": [bbox]}, crs="EPSG:4326")

#  Step 1: Loop through all GeoTIFF files in the folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif"):
        input_raster = os.path.join(input_folder, filename)
        output_raster = os.path.join(output_folder, filename)

        with rasterio.open(input_raster) as src:
            bbox_proj = bbox_gdf.to_crs(src.crs)  # Convert bounding box to raster CRS
            out_image, out_transform = mask(src, bbox_proj.geometry, crop=True, nodata=np.nan)
            out_meta = src.meta.copy()

        #  Step 2: Save Cropped Raster
        out_meta.update({"driver": "GTiff", "height": out_image.shape[1],
                         "width": out_image.shape[2], "transform": out_transform})

        with rasterio.open(output_raster, "w", **out_meta) as dst:
            dst.write(out_image)

        print(f" Cropped: {filename}")

print(" All Residual AOD Files Cropped Successfully!")


In [ ]:

#  Define Paths
base_dir = "D:/Sushrut/2024/Cropped_AOD_Files"  # Directory containing residual AOD files
output_dir = "D:/Sushrut/2024/High_Quality_Files"  # Destination folder
os.makedirs(output_dir, exist_ok=True)

#  Find all residual AOD files
residual_files = sorted(glob(os.path.join(base_dir, "*.tif")))

#  Debugging: Track file counts
total_files = len(residual_files)
copied_files = 0
skipped_files = 0

print(f" Total Residual AOD Files Found: {total_files}")

#  Process each file
for file in residual_files:
    try:
        with rasterio.open(file) as src:
            data = src.read(1)  # Read the raster data
            total_pixels = data.size
            nan_pixels = np.count_nonzero(np.isnan(data))
            nan_percentage = (nan_pixels / total_pixels) * 100

        #  Copy file if NaN percentage is <50%
        if nan_percentage < 50:
            dest_path = os.path.join(output_dir, os.path.basename(file))
            shutil.copy2(file, dest_path)  # Copy file instead of moving
            copied_files += 1
            print(f" Copied: {file} | NaN%: {nan_percentage:.2f}")
        else:
            skipped_files += 1
            print(f"️ Skipped: {file} | NaN%: {nan_percentage:.2f}")

    except Exception as e:
        print(f" Error reading {file}: {e}")
        skipped_files += 1

#  Summary
print("\n **Final Report:**")
print(f" Successfully copied {copied_files} files with <50% NaN to {output_dir}.")
print(f"️ Skipped {skipped_files} files due to >50% NaN or errors.")



In [ ]:

#  Define Paths
high_quality_dir = "D:/Sushrut/2024/High_Quality_Files"  # High-quality files
output_dir = "D:/Sushrut/2024/Interpolated_AOD"  # Output directory
os.makedirs(output_dir, exist_ok=True)

#  Get all high-quality files
high_quality_files = sorted(glob(os.path.join(high_quality_dir, "*.tif")))

#  Function to Perform Bilinear Interpolation
def bilinear_interpolation(input_file, output_file):
    with rasterio.open(input_file) as src:
        data = src.read(1)  # Read the single-band raster
        meta = src.meta.copy()

    # Convert NaNs to OpenCV-compatible format (-9999 for missing values)
    data[np.isnan(data)] = -9999

    # Apply Bilinear Interpolation using OpenCV (INPAINT_TELEA for better edge recovery)
    interpolated_data = cv2.inpaint(data.astype(np.float32), (data == -9999).astype(np.uint8), 3, cv2.INPAINT_TELEA)

    # Restore NaN values to interpolated areas
    interpolated_data[data == -9999] = np.nan

    # Save the interpolated raster
    with rasterio.open(output_file, "w", **meta) as dst:
        dst.write(interpolated_data.astype(rasterio.float32), 1)

    print(f" Interpolated & Saved: {output_file}")

#  Process Each High-Quality File
for file in high_quality_files:
    output_file = os.path.join(output_dir, os.path.basename(file))
    bilinear_interpolation(file, output_file)

print("\n Spatial Interpolation Completed for High-Quality Files!")


In [ ]:
#  Define Paths
smoothed_trend_dir = "D:/Sushrut/2024/Smoothed_Trend_AOD"
output_dir = "D:/Sushrut/2024/Cropped_Smoothed_Trend_AOD"
os.makedirs(output_dir, exist_ok=True)

#  Bounding Box in EPSG:4326 (WGS84)
xmin, ymin = 77.8889473735543, 17.078092493133276
xmax, ymax = 78.77358882331039, 17.87442882026007851

#  Check CRS of Raster
sample_file = glob(os.path.join(smoothed_trend_dir, "*.tif"))[0]
with rasterio.open(sample_file) as src:
    raster_crs = src.crs
    raster_bounds = src.bounds

    # Convert Bounding Box to Match Raster CRS
    if raster_crs != "EPSG:4326":
        xmin, ymin, xmax, ymax = transform_bounds("EPSG:4326", raster_crs, xmin, ymin, xmax, ymax)

#  Create Bounding Box with Updated CRS
bbox = box(xmin, ymin, xmax, ymax)

#  Function to Crop GeoTIFF
def crop_geotiff(input_file, output_file, bounding_box):
    with rasterio.open(input_file) as src:
        # Crop using bounding box
        try:
            out_image, out_transform = mask(src, [bounding_box], crop=True, nodata=np.nan)
        except ValueError as e:
            print(f"️ Skipping {input_file}: {e}")
            return  # Skip files that don't intersect

        out_meta = src.meta.copy()
        out_meta.update({
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        # Save cropped raster
        with rasterio.open(output_file, "w", **out_meta) as dst:
            dst.write(out_image)

    print(f" Cropped & Saved: {output_file}")

#  Process Each Smoothed Trend AOD File
smoothed_trend_files = sorted(glob(os.path.join(smoothed_trend_dir, "*.tif")))

for file in smoothed_trend_files:
    output_file = os.path.join(output_dir, os.path.basename(file))
    crop_geotiff(file, output_file, bbox)

print("\n Cropping Completed for Smoothed Trend AOD Files!")

In [ ]:


#  Base Directory
base_dir = "D:/Sushrut"
year = "2020"  # Process only for 2020

print(f"\n Processing Year: {year}...")

#  Define paths
high_quality_dir = os.path.join(base_dir, year, "High_Quality_Files")
smoothed_trend_dir = os.path.join(base_dir, year, "Cropped_Smoothed_Trend_AOD")
final_smoothed_dir = os.path.join(base_dir, year, "Final_Smoothed_AOD")

os.makedirs(final_smoothed_dir, exist_ok=True)

#  Step 1: Extract Dates from High-Quality Files
high_quality_files = glob(os.path.join(high_quality_dir, "*.tif"))

high_quality_dates = set()
for file in high_quality_files:
    filename = os.path.basename(file)
    if "_A" in filename:
        date_part = filename.split("_A")[1].split(".tif")[0]  # Extract YYYYDDD
        if date_part.isdigit():  # Ensure it's a valid date
            high_quality_dates.add(date_part)

print(f"    High-Quality Files Found: {len(high_quality_files)}")
print(f"    Unique Dates in High-Quality Files: {sorted(high_quality_dates)}")

#  Step 2: Copy Matching Smoothed AOD Files
smoothed_files = glob(os.path.join(smoothed_trend_dir, "*.tif"))
copied_files = 0

for file in smoothed_files:
    file_date = os.path.basename(file).split("_A")[1].split(".tif")[0]

    if file_date in high_quality_dates:  #  Only copy if date exists in High-Quality Files
        shutil.copy(file, os.path.join(final_smoothed_dir, os.path.basename(file)))
        copied_files += 1
    else:
        print(f"    Skipping {os.path.basename(file)} (No Matching High-Quality File)")

print(f"\n Copied {copied_files} Smoothed Trend AOD files to {final_smoothed_dir}.")
print("\n Process Complete for Year 2020!")


In [ ]:
# User Parameters
base_dir = "D:/Sushrut"
year = "2024"  # Change year if needed

# Input folders
smoothed_folder = os.path.join(base_dir, year, "Final_Smoothed_AOD")
residual_folder = os.path.join(base_dir, year, "Interpolated_AOD")

# Output folder
output_folder = os.path.join(base_dir, year, "Processed_GapFilled_AOD")
os.makedirs(output_folder, exist_ok=True)

# Get all input files sorted by date
smoothed_files = sorted(glob(os.path.join(smoothed_folder, "*.tif")))
residual_files = sorted(glob(os.path.join(residual_folder, "*.tif")))

# Check matching counts
if len(smoothed_files) != len(residual_files):
    print("Warning: Number of smoothed and residual files do not match!")
    print(f"Smoothed: {len(smoothed_files)}, Residuals: {len(residual_files)}")

# Combine and save
for smooth_file, residual_file in zip(smoothed_files, residual_files):
    # Extract date string from file
    date_str = os.path.basename(smooth_file).split("_A")[1].split(".tif")[0]

    # Read rasters
    with rasterio.open(smooth_file) as src_smooth:
        smooth_data = src_smooth.read(1).astype(np.float32)
        meta = src_smooth.meta.copy()

    with rasterio.open(residual_file) as src_res:
        residual_data = src_res.read(1).astype(np.float32)

    # Compute Final Gap-Filled AOD
    gapfilled_data = smooth_data + residual_data

    # Clip to realistic range (0–5)
    gapfilled_data = np.clip(gapfilled_data, 0, 5)

    # Update metadata and save
    meta.update(dtype=rasterio.float32, nodata=np.nan)
    output_file = os.path.join(output_folder, f"GapFilled_AOD_A{date_str}.tif")

    with rasterio.open(output_file, "w", **meta) as dst:
        dst.write(gapfilled_data, 1)

    print(f"Saved: {output_file}")

print("All Final Gap-Filled AOD files generated.")

In [ ]:


# Base Directory
base_dir = "C:/SushrutK"
years = ["2020", "2021", "2022", "2023", "2024"]

# Sampling Locations (Geographic CRS: EPSG:4326)
locations = {
    "Bollaram_Industrial Area": (78.358528, 17.540891),
    "Hyderabad Central University": (78.334361, 17.460103),
    "US Consulate": (78.3310916, 17.4248562),
    "ICRISAT": (78.278777, 17.518400),
    "Kokapet": (78.339194, 17.393559),
    "Sanathnagar": (78.4439295, 17.4562544),
    "Somajiguda": (78.457437, 17.417094),
    "Zoo Park, Bahadurpura West": (78.451437, 17.349694),
    "New Malkapet": (78.50864, 17.37206),
    "IIT, Hyderabad": (78.126199, 17.585705),
    "Nacharam TSIIC-IALA": (78.569354, 17.429398),
    "ECII- Kapra, Hyderabad": (78.566959, 17.470431),
    "Komapally Municipal Office": (78.486949, 17.544899),
    "Ramachandrapuram": (78.286195, 17.528544),
    "Pashamylaram": (78.218939, 17.5316895),
}

# Initialize dictionary to store data
data_dict = {}

# Process Each Year
for year in years:
    input_folder = os.path.join(base_dir, year, "Processed_GapFilled_AOD")
    tiff_files = sorted(glob(os.path.join(input_folder, "GapFilled_AOD_*.tif")))

    for tiff in tiff_files:
        # Extract date from filename
        date_str = os.path.basename(tiff).split("_")[-1].split(".tif")[0]  # Extract YYYYDDD

        with rasterio.open(tiff) as src:
            raster_crs = src.crs  # Get raster CRS

            # Convert Geographic Coordinates to Raster CRS
            lons, lats = zip(*locations.values())
            utm_x, utm_y = transform("EPSG:4326", raster_crs.to_string(), lons, lats)

            # Initialize data for this date if not already present
            if date_str not in data_dict:
                data_dict[date_str] = {loc: None for loc in locations.keys()}

            # Extract AOD Values
            for loc_name, x, y in zip(locations.keys(), utm_x, utm_y):
                try:
                    row, col = src.index(x, y)  # Get row and col indices
                    aod_value = src.read(1)[row, col]  # Read AOD Value
                except IndexError:
                    aod_value = None  # If outside bounds

                # Store the AOD value for this location and date
                data_dict[date_str][loc_name] = aod_value

# Create DataFrame from extracted data
dates = sorted(data_dict.keys())
columns = ["Date"] + list(locations.keys())
formatted_data = pd.DataFrame(columns=columns)

for date in dates:
    row = {"Date": date}
    row.update(data_dict[date])
    formatted_data = pd.concat([formatted_data, pd.DataFrame([row])], ignore_index=True)

# Save DataFrame to CSV
output_csv_path = os.path.join(base_dir, "AOD_Extracted_Values27.csv")
os.makedirs(base_dir, exist_ok=True)
formatted_data.to_csv(output_csv_path, index=False)

print(f" AOD values saved to: {output_csv_path}")
